# Pika-analyysi Exceliin

Frekvenssitaulukot, ristiintaulukoinnit, tilastolliset tunnusluvut ja korrelaatiot Excel-tiedostoon.

Pika-analyysiä voit käyttää seuraavasti:

* Vaihda avattavan datan tilalle oma data
* Suorita koodi
* Koodin suorittamisen jälkeen Excel-tiedosto **pika.xlsx** löytyy samasta kansiosta kuin tämä muistio.

In [1]:
import pandas as pd

# Jos muuttujalla on ainutkertaisia arvoja yli RAJAn, niin muuttuja tulkitaan määrälliseksi
# RAJA-arvoa voit tarvittaessa muuttaa
RAJA = 10

# Vaihda tähän analysoitava data
df = pd.read_excel('https://taanila.fi/titanic.xlsx')

In [4]:
# Alustavat toimet

# Luon Listat määrällisistä ja kategorisista muuttujista
kvantit = []
kategoriset = []
for muuttuja in df:
    if (len(df[muuttuja].unique())>RAJA) & (df[muuttuja].dtype in ['int64', 'float64']):
        kvantit.append(muuttuja)
    elif len(df[muuttuja].unique())<=RAJA:
        kategoriset.append(muuttuja)

# Luon ExcelWriter-olion, joka kirjoittaa pika.xlsx-tiedostoon
writer = pd.ExcelWriter('pika.xlsx', engine='xlsxwriter')

# Muotoilut
percent_format = writer.book.add_format({'num_format':'0.0 %'})
header_format = writer.book.add_format({'bold': True, 'align': 'right', 'bottom': 1})
title_format = writer.book.add_format({'bold': True, 'align': 'left', 'bottom':1})
index_format = writer.book.add_format({'bold': True, 'align': 'left', 'border':0})

# Frekvenssitaulukot 

if kategoriset:
    # Pidän kirjaa Excelin rivinumerosta rivi-muuttujan avulla
    rivi = 0
    # Käyn for-silmukalla läpi kaikki kategoriset muuttujat
    for muuttuja in kategoriset:
        df1 = pd.crosstab(df[muuttuja], 'f')
        df1['%'] = df1/df1.sum()
        df1.loc['yhteensä'] = df1.sum()
        # Kirjoitan frekvenssitaulukon Excel-tiedoston Frekvenssit-taulukkovälilehdelle
        df1.to_excel(writer, sheet_name='Frekvenssit', startrow=rivi)
        
        # Muotoilut
        for col_num, value in enumerate(df1.columns.values):
            writer.sheets['Frekvenssit'].write(rivi, col_num + 1, value, header_format)
        for ind_num, value in enumerate(df1.index.values):
            writer.sheets['Frekvenssit'].write(rivi+ind_num+1 , 0, value, index_format)
        writer.sheets['Frekvenssit'].write(rivi, 0, muuttuja, title_format)
    
        # Kasvatan rivinumeroa; shape[0] antaa df1:n rivimäärän
        rivi = rivi + df1.shape[0] + 2
    
    # Lisään C-sarakkeeseen prosenttimuotoilun
    writer.sheets['Frekvenssit'].set_column('C:C', cell_format=percent_format)
    
# Ristiintaulukoinnit
    
if len(kategoriset) > 1:
    rivi = 0
    for muuttuja1 in kategoriset:
        for muuttuja2 in kategoriset:
            if muuttuja1 != muuttuja2:
                df1 = pd.crosstab(df[muuttuja1], df[muuttuja2])
                df2 = pd.crosstab(df[muuttuja1], df[muuttuja2], normalize='columns')
                df2.index.name=muuttuja1+'/'+muuttuja2 
                df2.loc['n'] = df1.sum()
                df2.to_excel(writer, sheet_name='Ristiintaulukoinnit', startrow=rivi)
                                
                # Muotoilut
                for col_num, value in enumerate(df2.columns.values):
                    writer.sheets['Ristiintaulukoinnit'].write(rivi, col_num + 1, value, header_format)
                for ind_num, value in enumerate(df2.index.values):
                    writer.sheets['Ristiintaulukoinnit'].write(rivi+ind_num+1 , 0, value, index_format)
                writer.sheets['Ristiintaulukoinnit'].write(rivi, 0, muuttuja1+'/'+muuttuja2, title_format)
                for i in range(rivi+1, rivi+df2.shape[0]):
                    writer.sheets['Ristiintaulukoinnit'].set_row(i, cell_format=percent_format)
                
                rivi = rivi+df2.shape[0]+2
        
# Tunnusluvut
            
if kvantit:
    df1 = df[kvantit].describe()
    df1.to_excel(writer, sheet_name='Tunnusluvut)
    
    # Muotoilut
    for col_num, value in enumerate(df1.columns.values):
        writer.sheets['Tunnusluvut'].write(0, col_num + 1, value, header_format)
    for ind_num, value in enumerate(df1.index.values):
        writer.sheets['Tunnusluvut'].write(ind_num+1 , 0, value, index_format)
    writer.sheets['Tunnusluvut'].write(0, 0, muuttuja1+'/'+muuttuja2, title_format)
    
    # Tunnusluvut kategoristen määräämissä ryhmissä
    if kategoriset:
        rivi = df1.shape[0]+2
        for muuttuja1 in kategoriset:
            for muuttuja2 in kvantit:
                if muuttuja1 != muuttuja2:
                    df1 = df.groupby(muuttuja1)[muuttuja2].describe()
                    df1.index.name = muuttuja1+'/'+muuttuja2
                    df1.to_excel(writer, sheet_name='Tunnusluvut', startrow = rivi)
                    
                    # Muotoilut
                    for col_num, value in enumerate(df1.columns.values):
                        writer.sheets['Tunnusluvut'].write(rivi, col_num + 1, value, header_format)
                    for ind_num, value in enumerate(df1.index.values):
                        writer.sheets['Tunnusluvut'].write(rivi+ind_num+1 , 0, value, index_format)
                    writer.sheets['Tunnusluvut'].write(rivi, 0, muuttuja1+'/'+muuttuja2, title_format)

                    rivi = rivi + df1.shape[0]+2

# Korrelaatiot                
                
if kvantit:
    df1 = df[kvantit].corr()
    df1.to_excel(writer, sheet_name='Korrelaatiot', float_format='%.3f', header=False)
    
    # Muotoilut
    for col_num, value in enumerate(df1.columns.values):
        writer.sheets['Korrelaatiot'].write(0, col_num + 1, value, header_format)
    for ind_num, value in enumerate(df1.index.values):
        writer.sheets['Korrelaatiot'].write(ind_num+1 , 0, value, index_format)
    writer.sheets['Korrelaatiot'].write(rivi, 0, '', title_format)
    

# Excel-tiedoston tallennus
writer.close()

Taulukon otsikoiden muotoilu Excelissä on konstikasta. Katso https://xlsxwriter.readthedocs.io/working_with_pandas.html#formatting-of-the-dataframe-headers

Lisätietoa Pythonin käytöstä data-analytiikassa https://tilastoapu.wordpress.com/python/